In [1]:
from dotenv import load_dotenv, find_dotenv

# read local .env file
# find_dotenv 函数通常用于搜索和定位目录树中的 .env 文件，而 load_dotenv 函数用于将 .env 文件中的变量加载到环境中。
_ = load_dotenv(find_dotenv())
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.3)

In [2]:
"""
ConversationSummaryBufferMemory将最近的交互记录缓存在内存中，但不仅仅是完全清除旧的交互，而是将它们编译成一份摘要并同时使用。
不过，与之前的实现不同，它使用令牌长度而不是交互数量来确定何时清除交互。
"""
from langchain.memory import ConversationSummaryBufferMemory
memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=10)
memory.save_context({"input": "hi"}, {"output": "whats up"})
memory.save_context({"input": "not much you"}, {"output": "not much"})
 

In [3]:
memory.load_memory_variables({})
 

{'history': 'System: The human greets the AI and the AI asks what is going on. The human responds by saying "not much, you?"\nAI: not much'}

In [ ]:
""" output
{'history': 'System: The human greets the AI and the AI asks what is going on. The human responds by saying "not much, you?"\nAI: not much'}
"""

In [4]:
"""
我们还可以将历史记录作为消息列表获取（如果您正在使用聊天模型，则此功能很有用)。
"""

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=10, return_messages=True)
memory.save_context({"input": "hi"}, {"output": "whats up"})
memory.save_context({"input": "not much you"}, {"output": "not much"})
 

In [5]:
"""
我们还可以直接利用predict_new_summary方法。
"""
messages = memory.chat_memory.messages
previous_summary = ""
memory.predict_new_summary(messages, previous_summary)
 

"The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential. The AI also mentions that there isn't much else to say about it."

In [ ]:
""" output
"The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential. The AI also mentions that there isn't much else to say about it."
"""

In [6]:
"""
在链式中使用
"""
from langchain.chains import ConversationChain
conversation_with_summary = ConversationChain(
    llm=llm, 
    # We set a very low max_token_limit for the purposes of testing.
    memory=ConversationSummaryBufferMemory(llm=llm, max_token_limit=40),
    verbose=True
)
conversation_with_summary.predict(input="Hi, what's up?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, what's up?
AI:

> Finished chain.


"Hello! I'm an AI designed to have conversations and provide information. Right now, I'm here to chat with you. How can I assist you today?"

In [ ]:
""" output


[1m> Entering new ConversationChain chain...[0m
Prompt after formatting:
[32;1m[1;3mThe following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, what's up?
AI:[0m

[1m> Finished chain.[0m

"Hello! I'm an AI designed to have conversations and provide information. Right now, I'm here to chat with you. How can I assist you today?"
"""

In [7]:
conversation_with_summary.predict(input="Just working on writing some documentation!")




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human greets the AI and asks what's up.
AI: Hello! I'm an AI designed to have conversations and provide information. Right now, I'm here to chat with you. How can I assist you today?
Human: Just working on writing some documentation!
AI:

> Finished chain.


"That's great! Documentation is an important part of any project. What kind of documentation are you working on? Is it for a specific software or process?"

In [ ]:
""" output


[1m> Entering new ConversationChain chain...[0m
Prompt after formatting:
[32;1m[1;3mThe following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human greets the AI and asks what's up.
AI: Hello! I'm an AI designed to have conversations and provide information. Right now, I'm here to chat with you. How can I assist you today?
Human: Just working on writing some documentation!
AI:[0m

[1m> Finished chain.[0m

"That's great! Documentation is an important part of any project. What kind of documentation are you working on? Is it for a specific software or process?"
"""

In [8]:
# We can see here that there is a summary of the conversation and then some previous interactions
conversation_with_summary.predict(input="For LangChain! Have you heard of it?")
 



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human greets the AI and asks what's up. The AI responds by introducing itself as an AI designed for conversations and information. The human mentions that they are currently working on writing documentation.
AI: That's great! Documentation is an important part of any project. What kind of documentation are you working on? Is it for a specific software or process?
Human: For LangChain! Have you heard of it?
AI:

> Finished chain.


"Yes, I have heard of LangChain! It is a blockchain platform specifically designed for language-related applications. It aims to provide a decentralized and secure environment for language-related services such as translation, interpretation, and language learning. LangChain utilizes smart contracts and a native cryptocurrency called LangCoin to facilitate transactions and incentivize participants. It also incorporates machine learning and natural language processing technologies to enhance the efficiency and accuracy of language-related tasks. It's an interesting project! Is there anything specific you would like to know about LangChain?"

In [ ]:
""" output


[1m> Entering new ConversationChain chain...[0m
Prompt after formatting:
[32;1m[1;3mThe following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human greets the AI and asks what's up. The AI responds by introducing itself as an AI designed for conversations and information. The human mentions that they are currently working on writing documentation.
AI: That's great! Documentation is an important part of any project. What kind of documentation are you working on? Is it for a specific software or process?
Human: For LangChain! Have you heard of it?
AI:[0m

[1m> Finished chain.[0m

"Yes, I have heard of LangChain! It is a blockchain platform specifically designed for language-related applications. It aims to provide a decentralized and secure environment for language-related services such as translation, interpretation, and language learning. LangChain utilizes smart contracts and a native cryptocurrency called LangCoin to facilitate transactions and incentivize participants. It also incorporates machine learning and natural language processing technologies to enhance the efficiency and accuracy of language-related tasks. It's an interesting project! Is there anything specific you would like to know about LangChain?"
"""

In [9]:
# We can see here that the summary and the buffer are updated
conversation_with_summary.predict(input="Haha nope, although a lot of people confuse it for that")
 



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human greets the AI and asks what's up. The AI responds by introducing itself as an AI designed for conversations and information. The human mentions that they are currently working on writing documentation for LangChain, a blockchain platform for language-related applications. The AI is familiar with LangChain and explains its features, such as smart contracts, a native cryptocurrency called LangCoin, and the incorporation of machine learning and natural language processing technologies. The AI expresses interest in learning more about LangChain and asks if there are any specific details the human would like to know.
Human: Haha nope, although a lot of peo

'Confuse what for what? Could you please clarify?'

In [ ]:
""" output


[1m> Entering new ConversationChain chain...[0m
Prompt after formatting:
[32;1m[1;3mThe following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human greets the AI and asks what's up. The AI responds by introducing itself as an AI designed for conversations and information. The human mentions that they are currently working on writing documentation for LangChain, a blockchain platform for language-related applications. The AI is familiar with LangChain and explains its features, such as smart contracts, a native cryptocurrency called LangCoin, and the incorporation of machine learning and natural language processing technologies. The AI expresses interest in learning more about LangChain and asks if there are any specific details the human would like to know.
Human: Haha nope, although a lot of people confuse it for that
AI:[0m

[1m> Finished chain.[0m

'Confuse what for what? Could you please clarify?'
"""